In [13]:
import csv
import re
import sys
import time
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import metrics, tree
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

In [14]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str, encoding = 'utf-8',
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])
# df = shuffle(df)

In [15]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [ ]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
#     punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
#     no_punct = ""
#     for char in sample:
#         if char not in punctuations:
#             no_punct = no_punct + char
#     return no_punct
    return re.sub(r'[^\w\s\&\#\@\$\%\_]','',sample)

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au')]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = [w for w in sample.split(' ') if len(w) >= 2]
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = [w for w in sample.split(' ') if len(w) >= 2]
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()

def lemmy(sample):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    words = [w for w in sample.split(' ') if len(w) >= 2]
    lemmed_text = ""
    for word in words:
        lemmed_text = lemmed_text + lemmatizer.lemmatize(word) + " "
    return lemmed_text.rstrip()
    
def snowball(sample):
    words = [w for w in sample.split(' ') if len(w) >= 2]
    stemmer = SnowballStemmer("english")
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + stemmer.stem(word) + " "
    return stemmed_text.rstrip()

def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
    sample = lemmy(sample)
    sample = snowball(sample)
    sample = porter_stem(sample)
    return sample


In [ ]:
#myTokenizer('While the majority of all Internet search engines utilize stop words, they do not prevent a user from using them, but they are ignored.')

In [ ]:
# try to use sklearn stop_words later
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=800, encoding = 'utf-8')
# count = TfidfVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=200, encoding = 'utf-8', use_idf=True)
bag_of_words = count.fit_transform(text_data)
# print(count.get_feature_names()
# size = len(count.vocabulary_)
print(len(count.vocabulary_))

In [ ]:
X = bag_of_words.toarray()
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)
# First 1500 for training set, last 500 for test set
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)

In [ ]:
clf = MultinomialNB()
# clf = BernoulliNB()
model = clf.fit(X_train, y_train)
text_clf_red = Pipeline([('vect', CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer)), 
                       ('reducer', SelectKBest(chi2, k=800)),
                       ('clf', MultinomialNB())
                       ])
model_new = text_clf_red.fit(text_data[:1500],Y[:1500])

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

In [ ]:
y_pred = model_new.predict(text_data[1500:])
print(classification_report(Y[1500:], y_pred))

In [ ]:
y_pred = model_new.predict(text_data[:1500])
print(classification_report(Y[:1500], y_pred))